Importar las librerias que se necesitan

In [3]:
import pandas as pd
import os
import glob
import yfinance as yf
from datetime import datetime
import math
import numpy as np

Descargar los datos necesarios desde Yahoo Finance (se puede automatizar con airflow)

In [4]:
"""

## PARA DESCARGAR LA NUEVA INFORMACION - TARDÓ 120 Minutos, 87 Minutos 
# Ruta al directorio donde se encuentran los archivos CSV
directorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'

# Patrón para buscar archivos CSV
patron = '*.csv'

# Obtener la ruta de directorio de cada uno
archivos_csv = glob.glob(os.path.join(directorio, patron))

# Descargar los datos de Yahoo Finance y almacenarlos en un DataFrame por cada stock
for archivo_csv in archivos_csv:
    # Obtener el símbolo del stock desde el nombre del archivo
    nombre_archivo = os.path.basename(archivo_csv)
    simbolo = nombre_archivo.split('.')[0]  # Suponemos que el nombre del archivo es el símbolo del stock
    
    # Descargar los datos desde Yahoo Finance para los últimos 10 años
    data = yf.download(simbolo, start='2013-01-01')
    
    # Filtrar las columnas necesarias (Precio de apertura, cierre, cierre ajustado y volumen)
    data = data[['Open', 'Close', 'Adj Close', 'Volume']]
    
    # Guardar los datos en un nuevo archivo CSV
    nuevo_nombre_archivo = f'{simbolo}.csv'
    ruta_nuevo_archivo = os.path.join(directorio, nuevo_nombre_archivo)
    data.to_csv(ruta_nuevo_archivo)

"""

"\n\n## PARA DESCARGAR LA NUEVA INFORMACION - TARDÓ 120 Minutos, 87 Minutos \n# Ruta al directorio donde se encuentran los archivos CSV\ndirectorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'\n\n# Patrón para buscar archivos CSV\npatron = '*.csv'\n\n# Obtener la ruta de directorio de cada uno\narchivos_csv = glob.glob(os.path.join(directorio, patron))\n\n# Descargar los datos de Yahoo Finance y almacenarlos en un DataFrame por cada stock\nfor archivo_csv in archivos_csv:\n    # Obtener el símbolo del stock desde el nombre del archivo\n    nombre_archivo = os.path.basename(archivo_csv)\n    simbolo = nombre_archivo.split('.')[0]  # Suponemos que el nombre del archivo es el símbolo del stock\n    \n    # Descargar los datos desde Yahoo Finance para los últimos 10 años\n    data = yf.download(simbolo, start='2013-01-01')\n    \n    # Filtrar las columnas necesarias (Precio de apertura, cierre, cierre ajustado y volumen)\n    data = data[['Op

Se extraen los nombres de las acciones de NASDAQ y NYSE

In [5]:
# Ruta al directorio donde se encuentran los archivos CSV
directorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'

# Patrón para buscar archivos CSV
patron = '*.csv'

# Obtener la ruta de directorio de cada uno
archivos_csv = glob.glob(os.path.join(directorio, patron))

# Lista para almacenar los nombres de los archivos CSV
nombres_archivos_csv = []

# Extraer el nombre de cada archivo CSV
for archivo in archivos_csv:
    nombres_archivos_csv.append(os.path.basename(archivo))

Se crea un DataFrame por cada Stock

In [6]:
# Lista para almacenar los DataFrames de los archivos CSV
dataframes_csv = []
# Leer cada archivo CSV y almacenar el DataFrame correspondiente
for i in range(len(nombres_archivos_csv)):
    ruta_archivo = directorio + '/' + nombres_archivos_csv[i]  # Ruta completa al archivo
    df = pd.read_csv(ruta_archivo)
    dataframes_csv.append(df)

# Crea el DF por accion
for i in range(len(nombres_archivos_csv)):
    # Extraer un nombre de la lista
    nombre_variable = nombres_archivos_csv[i].replace('.csv','')
    # Asignar un valor a la variable con el nombre extraído
    globals()[nombre_variable] = dataframes_csv[i]

Se crea una lista con el nombre de cada stock (No la ruta del archivo)

In [7]:
# Extraer sólo el nombre del stock
for i in range(len(nombres_archivos_csv)):
    nombres_archivos_csv[i] = nombres_archivos_csv[i].replace('.csv','')

Se empiezan a eliminar los Stocks que no se pueden utilizar 

Se eliminan los df que no tengan datos, que no tengan histórico desde el primer día hábil del 2013, que no estén cotizando para el día hábil más cercano de cuando descargamos los datos, y que tengan información continua (que no falte ninguna cotización)

In [8]:
archivos_inutiles = []
length = len(globals()[nombres_archivos_csv[0]]['Date'])
ult_fecha = A['Date'][len(A['Date'])-1]

# Iterar sobre los nombres de archivos CSV
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]
    fecha = dataframe['Date']
    
    if len(fecha) != length or fecha[0] != '2013-01-02'  or fecha[len(fecha) - 1] != ult_fecha:
        archivos_inutiles.append(nombre_archivo)

# Eliminar los DataFrames inútiles y sus nombres de archivos CSV
for nombre_archivo in archivos_inutiles:
    nombre_dataframe = nombre_archivo.replace('.csv', '')
    del globals()[nombre_dataframe]
    nombres_archivos_csv.remove(nombre_archivo)

Calcular Desviación Estandard y rango de precios 

In [9]:
metrics = pd.DataFrame()

metrics['Stock'] = nombres_archivos_csv
std = []

for i in range(len(metrics['Stock'])):
    std.append(np.std(globals()[metrics['Stock'][i]]['Adj Close']))

metrics['Standard Deviation'] = std


In [10]:
r_min = []
r_max = []
fecha = '2023-07-14'    ## Fecha de 'Ayer', del dia anterior para basarse ena estimación del precio de "hoy" (Siempre tiene que ser de días hábiles)
indice = globals()[metrics['Stock'][0]].loc[A['Date'] == fecha].index[0]    ## Como todos los archivos tienen la misma cantidad de datos, si en uno se encuentra la fecha en x 
                                                                            ## posicion entonces en esa misma estará para todos los demás df
for i in range(len(metrics['Stock'])):
    r_min.append(globals()[metrics['Stock'][i]]['Close'][indice] - metrics['Standard Deviation'][i])
    r_max.append(globals()[metrics['Stock'][i]]['Close'][indice] + metrics['Standard Deviation'][i])

metrics['Rango Mínimo Estimado'] = r_min
metrics['Rango Máximo Estimado'] = r_max

Calcular rendimientos diarios, mensuales y anuales con su respectivo promedio

In [11]:
## PARA CALCULAR LOS RENDIMIENTOS DIARIOS -> (Pf/Pi)-1
for j in range(len(nombres_archivos_csv)):
    nombre_archivo = nombres_archivos_csv[j]
    dataframe = globals()[nombre_archivo]
    dataframe['Rendimientos diarios (%)'] = np.nan
    adj_close = dataframe['Adj Close']
    rendimientos = [((adj_close[i] / adj_close[i-1]) - 1)*100 for i in range(1, len(adj_close))]
    dataframe['Rendimientos diarios (%)'][1:] = rendimientos    

C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_14716\3329189856.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Rendimientos diarios (%)'][1:] = rendimientos
C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_14716\3329189856.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Rendimientos diarios (%)'][1:] = rendimientos
C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_14716\3329189856.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [12]:
## CALCULAR PROMEDIO DE RENDIMIENTOS DIARIOS POR STOCK
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(globals()[metrics['Stock'][i]]['Rendimientos diarios (%)']))
metrics['Rendimiento promedio diario (%)'] = p_rend

In [13]:
## CALCULAR PROMEDIO DE RENDIMIENTOS MENSUALES POR STOCK

# Crear un diccionario para almacenar los DataFrames con los rendimientos mensuales
rendimientos_mensuales = {}

# Calcular el rendimiento mensual para cada DataFrame
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]

    # Antes de aplicar resample, convertimos la columna 'Date' en un índice de fecha
    dataframe['Date'] = pd.to_datetime(dataframe['Date'])
    
    # Agrupar los datos por año y mes y obtener el primer y último valor de "Adj Close" de cada mes
    monthly_data = dataframe.resample('M', on='Date').agg({'Adj Close': ['first', 'last']})
    
    # Calcular el rendimiento del mes
    monthly_data['Rendimiento mensual (%)'] = (monthly_data['Adj Close']['last'] / monthly_data['Adj Close']['first'] - 1) * 100
    
    # Almacenar el DataFrame con los rendimientos mensuales en el diccionario
    rendimientos_mensuales[nombre_archivo] = monthly_data

# Ahora se puede acceder a los DataFrames individuales con sus nombres
# por ejemplo, dataframes_rendimientos_mensuales['NombreDelDataFrame']


In [14]:
## CALCULAR PROMEDIO DE RENDIMIENTOS MENSUALES POR STOCK
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(rendimientos_mensuales[metrics['Stock'][i]]['Rendimiento mensual (%)']))
metrics['Rendimiento promedio mensual (%)'] = p_rend

In [15]:
## CALCULAR PROMEDIO DE RENDIMIENTOS ANUALES POR STOCK

# Crear un diccionario para almacenar los DataFrames con los rendimientos mensuales
rendimientos_anuales = {}

# Calcular el rendimiento mensual para cada DataFrame
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]
    
    # Agrupar los datos por año y mes y obtener el primer y último valor de "Adj Close" de cada mes
    annual_data = dataframe.resample('A', on='Date').agg({'Adj Close': ['first', 'last']})
    
    # Calcular el rendimiento del mes
    annual_data['Rendimiento Anual (%)'] = (annual_data['Adj Close']['last'] / annual_data['Adj Close']['first'] - 1) * 100
    
    # Almacenar el DataFrame con los rendimientos mensuales en el diccionario
    rendimientos_anuales[nombre_archivo] = annual_data

# Ahora puedes acceder a los DataFrames individuales con sus nombres
# por ejemplo, dataframes_rendimientos_mensuales['NombreDelDataFrame']

In [16]:
## CALCULAR PROMEDIO DE RENDIMIENTOS ANUALES POR STOCK
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(rendimientos_anuales[metrics['Stock'][i]]['Rendimiento Anual (%)']))
metrics['Rendimiento promedio anual (%)'] = p_rend

Calcular volatilidad utilizando los métodos Historical Volatility (HV) y Realized Volatility (RV) 

In [17]:
## METODO "Historical Volatility (HV)" 
    # HV = √VAR * √N
    # VAR = SD/N

var = []
HV = []
dias_negociados = len(A['Date'])
for i in range(len(metrics['Stock'])):
    var.append((metrics['Standard Deviation'][i])/(dias_negociados))
    HV.append(math.sqrt(var[i]) * math.sqrt(dias_negociados))

In [31]:
## METODO "Realized Volatility (RV)" 
    # RV = √((∑SDR)/N)
    # SDR = (Daily return)^2
SDR = []  # Para agregar la suma de cada stock

for j in range(len(metrics['Stock'])):
    sdr = []  # Reiniciar la lista de SDR para cada stock
    for i in range(1, dias_negociados):
        sdr.append((globals()[metrics['Stock'][j]]['Rendimientos diarios (%)'][i]) ** 2)
    SDR.append(sum(sdr))

# Ahora calculamos la volatilidad (RV) para cada stock
RV = pd.DataFrame()
RV['Stock'] = metrics['Stock']
RV['Realized Volatility (RV)'] = [((sdr_sum / dias_negociados) ** 0.5) for sdr_sum in SDR]


    Baja volatilidad: Menos del 10% de desviación estándar.
    Volatilidad moderada: Entre el 10% y el 20% de desviación estándar.
    Alta volatilidad: Más del 20% de desviación estándar.

Calcular valor intrínseco 

    Primero hay que calcular:
        Terminal Value (TV) 
        Terminal Growth Rate (g)
        Retention Ratio ***

In [ ]:
archivos_inutiles = []
length = len(globals()[nombres_archivos_csv[0]]['Date'])

# Iterar sobre los nombres de archivos CSV
for nombre_archivo in nombres_archivos_csv:
    
    if len(fecha) != length or fecha[0] != '2013-01-02'  or fecha[len(fecha) - 1] != ult_fecha:
        archivos_inutiles.append(nombre_archivo)

# Eliminar los DataFrames inútiles y sus nombres de archivos CSV
for nombre_archivo in archivos_inutiles:
    nombre_dataframe = nombre_archivo.replace('.csv', '')
    del globals()[nombre_dataframe]
    nombres_archivos_csv.remove(nombre_archivo)

In [26]:
import requests
from bs4 import BeautifulSoup

def get_net_income_last_quarter(ticker_symbol):
    url = f"https://www.investing.com/equities/{ticker_symbol}-income-statement"
    
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # Buscamos la tabla de ingresos
        table = soup.find("table", {"class": "genTbl reportTbl"})
        if table:
            rows = table.find_all("tr")
            # La última fila generalmente contiene los datos del último trimestre
            last_quarter_row = rows[27]

            # Buscamos el Net Income en la última fila (segundo elemento de la fila)
            net_income = last_quarter_row.find_all("td")[1].get_text(strip=True)
            return net_income

        else:
            print("No se encontró información de ingresos trimestrales.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud: {e}")
        return None

# Ejemplo de uso
ticker_symbol = 'american-airlines-group'  # Reemplaza 'AAPL' con el símbolo de la empresa que deseas consultar
net_income_last_quarter = get_net_income_last_quarter(ticker_symbol)
if net_income_last_quarter:
    print(f"Net Income del último trimestre de {ticker_symbol}: {net_income_last_quarter}")

Net Income del último trimestre de american-airlines-group: 1338


<function len(obj, /)>

In [27]:
len(metrics['Stock'])

2884

## EXPERIMENTOS DE WEB SCRAPPING

In [20]:
import requests
from bs4 import BeautifulSoup

def scrape_net_income(ticker_symbol):
    url = f'https://www.google.com/finance/quote/{ticker_symbol}:NASDAQ'
    
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # Encontrar la tabla de "Income Statement" por la etiqueta <span> que contiene el texto "Income Statement"
        income_statement_section = soup.find('span', string='Income Statement')
        if income_statement_section:
            # Buscar la tabla que sigue inmediatamente después de la sección de "Income Statement"
            income_statement_table = income_statement_section.find_next('table')
            print(income_statement_table)
            # Buscar la fila que contiene "Net Income" en la tabla de ingresos
            net_income_row = income_statement_table.find('td', string='Net Income')
            print(net_income_row)
            if net_income_row:
                # Obtener el valor del net income
                net_income_value = net_income_row.find_next('td').text
                return net_income_value
            else:
                return "No se encontró información de Net Income."

        else:
            return "No se encontró información de Income Statement."

    except requests.exceptions.HTTPError:
        return "Error al realizar la solicitud. Verifica el ticker symbol o inténtalo más tarde."

# Ejemplo de uso con el ticker symbol "GOOGL" de Alphabet Inc.
ticker_symbol = "GOOGL"
net_income = scrape_net_income(ticker_symbol)
print(f"Net Income de {ticker_symbol}: {net_income}")


<table class="slpEwd"><tr class="roXhBd"><th class="yNnsfe PFjsMe">(USD)</th><th class="yNnsfe">Jun 2023<span data-is-tooltip-wrapper="true"><i aria-describedby="i26" aria-hidden="true" class="google-material-icons wajT3" data-tooltip-anchor-boundary-type="2" data-tooltip-x-position="2" jsaction="mouseenter:tfO1Yc; focus:AHmuwe; blur:O22p3e; mouseleave:JywGue; touchstart:p6p2H; touchend:yfqBxc;mlnRJb:fLiPzd;" jscontroller="e2jnoe">info</i><div aria-hidden="true" class="EY8ABd-OWXEXe-TAWMXe" id="i26" role="tooltip">Fiscal Q2 2023 ended 6/30/23. Reported on 7/24/23.</div></span></th><th class="yNnsfe">Y/Y change</th></tr><tr class="roXhBd"><td class="J9Jhg"><span data-is-tooltip-wrapper="true"><div aria-describedby="incomeStatementTT0" class="rsPbEe" data-tooltip-anchor-boundary-type="2" data-tooltip-x-position="2" jsaction="mouseenter:tfO1Yc; focus:AHmuwe; blur:O22p3e; mouseleave:JywGue; touchstart:p6p2H; touchend:yfqBxc;mlnRJb:fLiPzd;" jscontroller="e2jnoe">Revenue</div><div aria-hidde

In [21]:
import requests

# URL de la API del sitio web con tu clave de API
url = "https://api.financialmodelingprep.com/api/v3/income-statement/A?apikey=TU_CLAVE_DE_API"

try:
    # Realizar una solicitud GET a la API
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        data = response.json()

        # Encontrar el net income en los datos obtenidos (la estructura puede variar según la API)
        net_income = data[0]['netIncome']
        print("Net Income:", net_income)
    else:
        print("Error en la solicitud a la API:", response.status_code)
except requests.exceptions.RequestException as e:
    print("Error de conexión:", e)



Error de conexión: HTTPSConnectionPool(host='api.financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/income-statement/A?apikey=TU_CLAVE_DE_API (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000028F7466C9D0>, 'Connection to api.financialmodelingprep.com timed out. (connect timeout=None)'))
